In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from xgboost import XGBRFRegressor

#### Os dados abaixo são retirados de um respositório médico, sendo estes dados sobre a coluna lombar de pacientes que não possuem nenhuma problema na coluna, ou que possuem Hérnia de disco ou Espondilolistese (escorregamento vertebral), o modelo abaixo tenta prever os pacientes que não possuem problema vertebral ou que possuem algum desses 2 problemas

In [2]:
#importando os dados do tipo arff 

import warnings, requests, zipfile, io
warnings.simplefilter('ignore')
import pandas as pd
from scipy.io import arff

data = arff.loadarff('column_2C_weka.arff')
dados = pd.DataFrame(data[0])

In [3]:
dados['class'].value_counts(), dados.shape

(class
 b'Abnormal'    210
 b'Normal'      100
 Name: count, dtype: int64,
 (310, 7))

In [4]:
# trocando os dados da coluna class para 0 que irá representar um paciente Normal e 1 para paciente com algum tipo de problema, 
# sendo estes problemas de Espondilolistese ou Hérnia de disco
mapeamento_class = {b'Normal':0, b'Abnormal':1}
dados['class'] = dados['class'].replace(mapeamento_class)

In [5]:
#colocando a coluna de "target" (class) como primeira no conjunto de dados para facilitar o processamento pelo algoritmo
colunas = dados.columns.to_list()
colunas = colunas[-1:] + colunas[:-1]
dados = dados[colunas]
 
dados.head()

,class,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
0,1,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400
1,1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259
2,1,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317
3,1,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523
4,1,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501


In [6]:
#os códigos abaixo servem para dividir a massa de dados em conjuntos diferentes para treinamento e teste
massa_dados_base = dados.iloc[:, 1:]
coluna_alvos_originais = dados.iloc[:, :1]

In [7]:
#definindo as variaveis para receber os dados repartidos pela função de divisão dos dados
dados_treinamento_base, dados_test_base, dados_treinamento_alvos, dados_test_alvos = train_test_split(massa_dados_base, coluna_alvos_originais, random_state=42, test_size=0.2)

In [8]:
#verificando o tamanho dos dados de treinamento 
dados_treinamento_base.shape , dados_treinamento_alvos.shape

((248, 6), (248, 1))

In [9]:
#verificando o tamanho dos dados para test e validação dos resultados
dados_test_base.shape , dados_test_alvos.shape

((62, 6), (62, 1))

In [14]:
#criando uma instância da classe do modelo XGBoost para utiliza-la na medição da acurácia logo abaixo utilizando o metodo de classificação
classificador_xgb = xgb.XGBClassifier()

In [23]:
#importando a biblioteca cross validation(CV) para treinar qual a acurácia do modelo escolhido utilizando os
# dados da base de treinamento para realizar testes com diferentes ordenações desses dados em cada teste de treinamento
resultado_cv = cross_val_score(classificador_xgb,dados_treinamento_base, dados_treinamento_alvos, cv=4) 
resultado_cv

# o processo acima serve somente para testar a acurácia média do modelo utilizando a base de dados trabalhada para treinamento
# esse processo pode ser utilizado com diversos tipos de modelos(algoritmos) e verificar sua acurácia antes de testa-lo com 
# dados de testes utilizando funções de treinamento do modelo.

array([0.85483871, 0.87096774, 0.80645161, 0.83870968])

In [40]:
#realizando a média de acurácia(cross validation) realizada acima referente os diferentes testes realizados
media_cv = resultado_cv.mean()
print(" Desempenho de : {:.2f} %".format(media_cv*100))

 Desempenho de : 84.27 %


In [66]:
#alterando os hiper-parametros do algoritmos XGB para melhorar a precisão da acurácia a fim de 
# deixar o modelo pronto para um futuro treinamento com dados de test
tunagem1_xgb = xgb.XGBClassifier(max_depth=5, booster='dart', reg_lambda=1.5) 

resultado_cv_tunagem1 = cross_val_score(tunagem1_xgb, dados_treinamento_base, dados_treinamento_alvos, cv = 4).mean()
print(" Desempenho de : {:.2f} %".format(resultado_cv_tunagem1*100))

 Desempenho de : 86.29 %


In [67]:
#testando uma nova implementação do XGB para testar novamente uma melhora na acurácia
tunagem2_xgb = xgb.XGBClassifier(booster='gblinear')
resultado_cv_tunagem2 = cross_val_score(tunagem2_xgb, dados_treinamento_base, dados_treinamento_alvos, cv = 4).mean()
print('Desempenho de: {:.2f} %'.format(resultado_cv_tunagem2*100))

#para testar novas métricas de desempenho(tunagem) tente realizar a mudança de diferentes parâmetros do XGBoost, o site abaixo mostra
# alguns dos principais parâmetros que podem ser trabalhados: https://www.kaggle.com/code/prashant111/a-guide-on-xgboost-hyperparameters-tuning

Desempenho de: 82.66 %


In [68]:
#definindo o modelo campeão com a melhor acurácia
modelo_campeao = tunagem1_xgb

In [69]:
#realizando o treinamento oficial do modelo com os dados de treinamento
modelo_campeao.fit(dados_treinamento_base, dados_treinamento_alvos)

XGBClassifier(base_score=None, booster='dart', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [72]:
#realizando uma predição com os dados de validação após o treinamento do modelo campeão escolhido
predicao = modelo_campeao.predict(dados_test_base)
predicao

array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [81]:
#jogando o resultado para um data frame 
dados_resultado = pd.DataFrame(predicao, index= dados_test_alvos.index, columns=['Resultado'])
dados_resultado.head()

,Resultado
289,0
9,0
57,0
60,1
25,1


In [89]:
#calculando os acertos da predição
soma_acertos = (dados_test_alvos['class'] == dados_resultado['Resultado']).sum()
soma_acertos

48

In [90]:
#calculando a porcentagem de acertos de acordo com o base de dados resultado(acurácia)
qtd_resultados_alvos = len(dados_test_alvos)
porct_acuracia = 100 * (soma_acertos/qtd_resultados_alvos)
porct_acuracia

print('Desempenho final do treinamento de Classificação: {:.2f} %'.format(porct_acuracia))

Desempenho final do treinamento de Classificação: 77.42 %
